In [1]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("fatemehmehrparvar/obesity-levels")

# print("Path to dataset files:", path)

In [2]:
# import os
# os.listdir(path)

In [3]:
import pandas as pd
import fairtl as fl
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [4]:
df = pd.read_csv("/home/morning/.cache/kagglehub/datasets/fatemehmehrparvar/obesity-levels/versions/1/ObesityDataSet_raw_and_data_sinthetic.csv")

In [5]:
# df = pd.read_csv(f'{path}/ObesityDataSet_raw_and_data_sinthetic.csv')
# df.head()

In [6]:
df.shape

(2111, 17)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Age                             2111 non-null   float64
 1   Gender                          2111 non-null   object 
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   CALC                            2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   SCC                             2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  family_history_with_overweight  2111 non-null   object 
 12  FAF                             21

In [8]:
df['CALC'].unique()

array(['no', 'Sometimes', 'Frequently', 'Always'], dtype=object)

In [9]:
df['CALC'] = df['CALC'].map({'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3})
df['FAVC'] = df['FAVC'].map({'no': 0, 'yes': 1})
df['SCC'] = df['SCC'].map({'no': 0, 'yes': 1})
df['SMOKE'] = df['SMOKE'].map({'no': 0, 'yes': 1})
df['Gender'] = df['Gender'].map({'Female': 0, 'Male': 1})
df['family_history_with_overweight'] = df['family_history_with_overweight'].map({'no': 0, 'yes': 1})
df['CAEC'] = df['CAEC'].map({'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3})


In [10]:
df['MTRANS'].unique()

array(['Public_Transportation', 'Walking', 'Automobile', 'Motorbike',
       'Bike'], dtype=object)

In [11]:
enc = OneHotEncoder(categories='auto')
mtrans = df['MTRANS'].values.reshape(-1, 1)
enc.fit(mtrans)
new_features = enc.get_feature_names_out()
print(new_features)
new_mtrans = pd.DataFrame(enc.transform(mtrans).toarray())

['x0_Automobile' 'x0_Bike' 'x0_Motorbike' 'x0_Public_Transportation'
 'x0_Walking']


In [12]:
new_mtrans.columns = ['Automobile', 'Bike', 'Motorbike', 'Public_Transportation', 'Walking']

In [13]:
df = pd.concat([df, new_mtrans], axis=1)
df.drop('MTRANS', axis=1, inplace=True)
df.head()

,Age,Gender,Height,Weight,CALC,FAVC,FCVC,NCP,SCC,SMOKE,...,family_history_with_overweight,FAF,TUE,CAEC,NObeyesdad,Automobile,Bike,Motorbike,Public_Transportation,Walking
0,21.0,0,1.62,64.0,0,0,2.0,3.0,0,0,...,1,0.0,1.0,1,Normal_Weight,0.0,0.0,0.0,1.0,0.0
1,21.0,0,1.52,56.0,1,0,3.0,3.0,1,1,...,1,3.0,0.0,1,Normal_Weight,0.0,0.0,0.0,1.0,0.0
2,23.0,1,1.80,77.0,2,0,2.0,3.0,0,0,...,1,2.0,1.0,1,Normal_Weight,0.0,0.0,0.0,1.0,0.0
3,27.0,1,1.80,87.0,2,0,3.0,3.0,0,0,...,0,2.0,0.0,1,Overweight_Level_I,0.0,0.0,0.0,0.0,1.0
4,22.0,1,1.78,89.8,1,0,2.0,1.0,0,0,...,0,0.0,0.0,1,Overweight_Level_II,0.0,0.0,0.0,1.0,0.0


In [14]:
df['NObeyesdad'].unique()

array(['Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II',
       'Obesity_Type_I', 'Insufficient_Weight', 'Obesity_Type_II',
       'Obesity_Type_III'], dtype=object)

In [15]:
df['NObeyesdad'] = df['NObeyesdad'].map({'Insufficient_Weight':0, 'Normal_Weight': 1, 'Overweight_Level_I': 2, 'Overweight_Level_II': 3, 'Obesity_Type_I': 4, 'Obesity_Type_II': 5, 'Obesity_Type_III': 6})

In [16]:
df.head()

,Age,Gender,Height,Weight,CALC,FAVC,FCVC,NCP,SCC,SMOKE,...,family_history_with_overweight,FAF,TUE,CAEC,NObeyesdad,Automobile,Bike,Motorbike,Public_Transportation,Walking
0,21.0,0,1.62,64.0,0,0,2.0,3.0,0,0,...,1,0.0,1.0,1,1,0.0,0.0,0.0,1.0,0.0
1,21.0,0,1.52,56.0,1,0,3.0,3.0,1,1,...,1,3.0,0.0,1,1,0.0,0.0,0.0,1.0,0.0
2,23.0,1,1.80,77.0,2,0,2.0,3.0,0,0,...,1,2.0,1.0,1,1,0.0,0.0,0.0,1.0,0.0
3,27.0,1,1.80,87.0,2,0,3.0,3.0,0,0,...,0,2.0,0.0,1,2,0.0,0.0,0.0,0.0,1.0
4,22.0,1,1.78,89.8,1,0,2.0,1.0,0,0,...,0,0.0,0.0,1,3,0.0,0.0,0.0,1.0,0.0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Age                             2111 non-null   float64
 1   Gender                          2111 non-null   int64  
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   CALC                            2111 non-null   int64  
 5   FAVC                            2111 non-null   int64  
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   SCC                             2111 non-null   int64  
 9   SMOKE                           2111 non-null   int64  
 10  CH2O                            2111 non-null   float64
 11  family_history_with_overweight  2111 non-null   int64  
 12  FAF                             21

In [18]:
y = df['NObeyesdad'].values
X = df.drop('NObeyesdad', axis=1)

In [19]:
print(X.shape)
selector = VarianceThreshold(threshold=0.1)
X = selector.fit_transform(X)
print(X.shape)

(2111, 20)
(2111, 14)


In [20]:
# build mask
gender_0_mask = df['Gender'] == 0
gender_1_mask = df['Gender'] == 1

count_gender_0 = gender_0_mask.sum()
count_gender_1 = gender_1_mask.sum()

print("Female: ", count_gender_0)
print("Male: ", count_gender_1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_Gender_0 = X_scaled[gender_0_mask]
X_scaled_Gender_1 = X_scaled[gender_1_mask]
y_Gender_0 = y[gender_0_mask]
y_Gender_1 = y[gender_1_mask]

Female:  1043
Male:  1068


In [21]:
seed = 42
models = fl.build_models(seed)

results_list = []

kf = KFold(n_splits=10, shuffle=True, random_state=seed)

In [22]:
print("Starting experiments for Female(0)")
fl.run_experiment(kf, models, X_scaled_Gender_0, y_Gender_0, 'Female', results_list)

print("Starting experiments for Male(1)")
fl.run_experiment(kf, models, X_scaled_Gender_1, y_Gender_1, 'Male', results_list)

final_results_df = pd.concat(results_list, ignore_index=True)
print(final_results_df)

Starting experiments for Female(0)
Processing fold 1 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 2 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 3 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


...

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 9 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 10 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
    Fold   Group   SVM_TPR   SVM_TNR   SVM_FPR   SVM_FNR  SVM_TP  SVM_TN  \
0      1  Female  0.700000  1.000000  0.000000  0.300000      14      17   
1      2  Female  0.727273  0.944444  0.055556  0.272727       8      17   
2      3  Female  0.800000  0.920000  0.080000  0.200000       4      23   
3      4  Female  1.000000  1.000000  0.000000  0.000000      10      16   
4      5  Female  0.533333  0.928571  0.071429  0.466667       8      13   
5      6  Female  0.818182  0.950000  0.050000  0.181818       9      19   
6      7  Female  0.692308  1.000000  0.000000  0.307692       9      11   
7      8  Female  0.800000  0.900000  0.100000  0.200000       8      18   
8      

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [23]:
results_df = pd.concat(results_list, ignore_index=True)

In [24]:
result_path = './results/k143_result.xlsx'
results_df.to_excel(result_path, index=False)

In [25]:
df = pd.read_excel(result_path)
df.head()

,Fold,Group,SVM_TPR,SVM_TNR,SVM_FPR,SVM_FNR,SVM_TP,SVM_TN,SVM_FP,SVM_FN,...,ANN_FP,ANN_FN,NB_TPR,NB_TNR,NB_FPR,NB_FNR,NB_TP,NB_TN,NB_FP,NB_FN
0,1,Female,0.700000,1.000000,0.000000,0.300000,14,17,0,6,...,0,4,0.526316,1.000000,0.000000,0.473684,10,17,0,9
1,2,Female,0.727273,0.944444,0.055556,0.272727,8,17,1,3,...,1,3,0.400000,0.944444,0.055556,0.600000,4,17,1,6
2,3,Female,0.800000,0.920000,0.080000,0.200000,4,23,2,1,...,2,2,0.375000,1.000000,0.000000,0.625000,3,27,0,5
3,4,Female,1.000000,1.000000,0.000000,0.000000,10,16,0,0,...,0,1,0.666667,1.000000,0.000000,0.333333,6,16,0,3
4,5,Female,0.533333,0.928571,0.071429,0.466667,8,13,1,7,...,1,6,0.571429,1.000000,0.000000,0.428571,8,14,0,6


In [26]:
label = 'Female'

fl.perform_t_tests(df, 'SVM', label)
fl.perform_t_tests(df, 'DT', label)
fl.perform_t_tests(df, 'RF', label)
fl.perform_t_tests(df, 'LR', label)
fl.perform_t_tests(df, 'KNN', label)
fl.perform_t_tests(df, 'ANN', label)
fl.perform_t_tests(df, 'NB', label)


SVM -TPR: MannwhitneyuResult(statistic=np.float64(19.5), pvalue=np.float64(0.020350331290676295))
SVM - FPR: MannwhitneyuResult(statistic=np.float64(33.0), pvalue=np.float64(0.20569493140126405))
SVM - FN/FP: MannwhitneyuResult(statistic=np.float64(69.5), pvalue=np.float64(0.14192909303478157))
DT -TPR: TtestResult(statistic=np.float64(-0.2211434873879445), pvalue=np.float64(0.8274699253123258), df=np.float64(18.0))
DT - FPR: MannwhitneyuResult(statistic=np.float64(48.5), pvalue=np.float64(0.938675208112475))
DT - FN/FP: MannwhitneyuResult(statistic=np.float64(51.5), pvalue=np.float64(0.9387940283466353))
RF -TPR: MannwhitneyuResult(statistic=np.float64(42.0), pvalue=np.float64(0.45591013038912653))
RF - FPR: MannwhitneyuResult(statistic=np.float64(36.0), pvalue=np.float64(0.29327072669776244))
RF - FN/FP: MannwhitneyuResult(statistic=np.float64(56.0), pvalue=np.float64(0.5847793957835943))
LR -TPR: TtestResult(statistic=np.float64(-3.523930418873326), pvalue=np.float64(0.0024243904588